In [1]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [4]:
from huggingface_hub import login

# This will prompt you to enter your Hugging Face token securely.
login()

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Define the model name
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

print(f"Loading tokenizer for {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tokenizer loaded.")

print(f"Loading model for {model_name}...")
# Load the model in 4-bit to save memory
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True, # This is crucial for running on Colab's free GPU
    device_map="auto",
    torch_dtype=torch.float16 # Use float16 for better performance and lower memory usage
)
print("Model loaded.")

# Set the model to evaluation mode
model.eval()

Loading tokenizer for mistralai/Mistral-7B-Instruct-v0.2...


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Model loaded.


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

In [6]:
def get_health_response(query):
    # --- Prompt Engineering ---
    # System message to set the persona and guidelines
    system_message = (
        "You are a helpful and friendly medical assistant. "
        "You provide general health information and explanations, "
        "but **NEVER give specific medical advice, diagnoses, or prescribe treatments.** "
        "Always advise users to consult a qualified healthcare professional for personalized medical advice. "
        "Keep your answers clear, concise, and easy to understand. "
        "If a question asks for specific medical advice or a diagnosis, politely state that you cannot provide that and recommend seeing a doctor."
    )

    # User's query
    user_query = query

    # Combine system message and user query for the model
    # Mistral uses a specific format for its instructions, known as "instruction tuning"
    # The prompt template is important for getting good responses from Mistral
    prompt = f"<s>[INST] {system_message}\n\n{user_query} [/INST]"

    # --- Safety Filters (Rule-based for now) ---
    # These are basic, rule-based filters. For robust safety, more advanced techniques
    # like content moderation APIs or fine-tuning models on safety data are used.
    lower_query = query.lower()
    if "diagnose" in lower_query or \
       "prescribe" in lower_query or \
       "treatment for" in lower_query or \
       "should i take" in lower_query or \
       "what drug" in lower_query or \
       "am i having" in lower_query or \
       "is it cancer" in lower_query:
        return "I am a medical assistant and cannot provide medical advice, diagnoses, or prescribe treatments. Please consult a qualified healthcare professional for personalized medical guidance."

    # --- Generate Response from LLM ---
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    # Generate the response
    with torch.no_grad(): # Disable gradient calculation for inference, saves memory and speeds up
        outputs = model.generate(
            **inputs,
            max_new_tokens=500, # Maximum number of tokens to generate in the response
            temperature=0.7,    # Controls randomness: lower for more deterministic, higher for more creative
            do_sample=True,     # Enable sampling for more varied responses
            top_k=50,           # Consider only the top-k most likely tokens
            top_p=0.95,         # Nucleus sampling: consider tokens until cumulative probability reaches top_p
            pad_token_id=tokenizer.eos_token_id # Important for handling padding
        )

    # Decode the generated tokens back to text
    # We need to slice the output to remove the input prompt
    response_text = tokenizer.decode(outputs[0][len(inputs["input_ids"][0]):], skip_special_tokens=True)

    # --- Post-processing for safety and clarity ---
    # Add a general disclaimer at the end
    if "Please consult a qualified healthcare professional" not in response_text:
        response_text += "\n\n**Disclaimer:** This information is for general knowledge and informational purposes only, and does not constitute medical advice. Please consult a qualified healthcare professional for any health concerns or before making any decisions related to your health or treatment."

    return response_text.strip()

In [ ]:
print("Welcome to the General Health Chatbot! (Type 'quit' to exit)")

while True:
    user_input = input("\nYou: ")
    if user_input.lower() == 'quit':
        print("Chatbot: Goodbye!")
        break

    print("Chatbot (thinking...): ", end="")
    response = get_health_response(user_input)
    print(response)

Welcome to the General Health Chatbot! (Type 'quit' to exit)

You: What causes a sore throat?
Chatbot (thinking...): 

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:463: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


A sore throat is typically caused by infections such as the common cold or flu, strep throat, or other viruses. It can also be caused by environmental irritants like smoke or acid reflux. Other possible causes include bacterial infections, allergies, or the repeated use of your voice. It's essential to consult a healthcare professional for an accurate diagnosis and appropriate treatment.

**Disclaimer:** This information is for general knowledge and informational purposes only, and does not constitute medical advice. Please consult a qualified healthcare professional for any health concerns or before making any decisions related to your health or treatment.

You: Is paracetamol safe for children?
Chatbot (thinking...): Paracetamol, also known as acetaminophen, is commonly used to relieve fever and reduce pain in children. However, it's important to follow the recommended dosage instructions carefully. Overdose or incorrect usage can lead to potential health issues. Always consult the p